___
# Importing necessary libraries
___

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')


import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px


import cufflinks as cf
cf.go_offline()

In [46]:
# Loading the dataset
df = pd.read_csv('../dataset/COVID-19_Sentiments.csv')

In [47]:
df.head()

,Text_Id,Text,Date,Location,Sentiments
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,"Uttar Pradesh, India",0.000
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,Mumbai,0.000
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",Fri Mar 20 16:04:29 +0000 2020,"Jodhpur, India",0.350
3,1241032877099237379,RT @narendramodi: The young actors have someth...,Fri Mar 20 16:04:29 +0000 2020,"Gurugram, Bharat",0.125
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:28 +0000 2020,"New Delhi, India",0.000


In [48]:
# Retaining only date in Date feature
df['Date'] = pd.to_datetime( df['Date'] ).dt.date

In [49]:
df.head()

,Text_Id,Text,Date,Location,Sentiments
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350
3,1241032877099237379,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648958 entries, 0 to 648957
Data columns (total 5 columns):
Text_Id       648958 non-null int64
Text          648958 non-null object
Date          648958 non-null object
Location      648958 non-null object
Sentiments    637978 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 24.8+ MB


In [51]:
lis = []
for value in df.Sentiments:
    if value < 0 :
        lis.append(-1)
    elif value == 0 :
        lis.append(0)
    else:
        lis.append(1)
df['Sentiment Score'] = lis 
df.head()

,Text_Id,Text,Date,Location,Sentiments,Sentiment Score
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,1241032877099237379,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


In [52]:
# Getting data of only first phase of lockdown
import datetime
phase_first = df[ (df.Date >= datetime.date(2020, 3, 25)) & (df.Date <= datetime.date(2020, 4, 14)) ]

In [53]:
phase_first

,Text_Id,Text,Date,Location,Sentiments,Sentiment Score
23828,1242693324563046400,"RT @chathootti: Even this national emergency, ...",2020-03-25,"Tamil Nadu, India",NaN,1
23829,1242693299413995523,Now the slogan has been justified................,2020-03-25,India,NaN,1
23830,1242693303897743366,RT @Shehzad_Ind: Thank you @narendramodi @Amit...,2020-03-25,"Hindustan, India",NaN,1
23831,1242693322373767168,RT @jvin_tootu: Idea in the time of Corona\n A...,2020-03-25,"Aluva, India",NaN,1
23832,1242693346197254147,RT @RajiniFollowers: Super Bro 🙌🙌🙌🙌 #RAJINIVoi...,2020-03-25,"Bengaluru South, India",NaN,1
...,...,...,...,...,...,...
491428,1249965076477382657,RT @UnderpaidBanke8: No Insurance cover\nNo Re...,2020-04-14,"Haryana, India",-0.178571,-1
491429,1249965122707058688,RT @iAnkurSingh: 21 days would've been suffici...,2020-04-14,India,-0.071429,-1
491430,1249965119968165890,RT @Sanjaythech1234: No Insurance cover\nNo Re...,2020-04-14,"Haryana, India",-0.178571,-1
491431,1249965079518306305,RT @IYC_UPWest: There can not be a better day ...,2020-04-14,"New Delhi, India",0.500000,1


In [54]:
# Counting tweets only for +ve , -ve and neutral tweets
positive = list(phase_first['Sentiment Score']).count( 1 )
negative = list(phase_first['Sentiment Score']).count(-1 )
neutral  = list(phase_first['Sentiment Score']).count( 0 )

lis = [['positive' , positive] , ['negative' , negative] , ['neutral' , neutral]]
distribution = pd.DataFrame(lis , columns = ['Category' , 'Count'])
distribution

,Category,Count
0,positive,115308
1,negative,54504
2,neutral,116440


In [55]:
px.pie( distribution , values='Count', names='Category', title='First phase Tweets Category distribution' ,color_discrete_sequence=px.colors.sequential.RdBu)

In [56]:
df.head()

,Text_Id,Text,Date,Location,Sentiments,Sentiment Score
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,1241032877099237379,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648958 entries, 0 to 648957
Data columns (total 6 columns):
Text_Id            648958 non-null int64
Text               648958 non-null object
Date               648958 non-null object
Location           648958 non-null object
Sentiments         637978 non-null float64
Sentiment Score    648958 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 29.7+ MB


In [110]:
df1 = df.drop(['Text', 'Sentiments', 'Location'], axis = 1)

In [111]:
df1.head()

,Text_Id,Date,Sentiment Score
0,1241032866567356417,2020-03-20,0
1,1241032867699765249,2020-03-20,0
2,1241032875102703616,2020-03-20,1
3,1241032877099237379,2020-03-20,1
4,1241032870405128192,2020-03-20,0


In [112]:
df1['Date'] = pd.to_datetime( df1['Date'] )

In [113]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648958 entries, 0 to 648957
Data columns (total 3 columns):
Text_Id            648958 non-null int64
Date               648958 non-null datetime64[ns]
Sentiment Score    648958 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 14.9 MB


In [114]:
df1['month']  = df1['Date'].dt.month
df1['day'] = df1['Date'].dt.day

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

def different_models_performance(x , y):
    classifier = LogisticRegression()
    classifier.fit(x, y)
    print("Score for Logistic Regression : " , classifier.score(x,y))


    classifier = DecisionTreeClassifier()
    classifier.fit(x, y)
    print("Score for DecisionTreeClassifier : " , classifier.score(x,y))


    classifier = RandomForestClassifier()
    classifier.fit(x, y)
    print("Score for RandomForestClassifier : " , classifier.score(x,y))

In [116]:
    x = df1[['month', 'day']]
    y = df1['Sentiment Score']
    different_models_performance(x , y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Score for Logistic Regression :  0.40418023970734623
Score for DecisionTreeClassifier :  0.4470150610671261


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Score for RandomForestClassifier :  0.4469749968410899


In [117]:
    x = df1[['month', 'day' , 'Text_Id']]
    y = df1['Sentiment Score']
    different_models_performance(x , y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Score for Logistic Regression :  0.41428567025909224
Score for DecisionTreeClassifier :  0.48648448743986517


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Score for RandomForestClassifier :  0.48540891706397027


In [123]:
df1['No. of words'] = df['Text']
df1['No. of words'] = df1['No. of words'].apply(lambda x : len(x.replace('\n' , ' ').split()))

In [124]:
df1.head()

,Text_Id,Date,Sentiment Score,month,day,No. of words
0,1241032866567356417,2020-03-20,0,3,20,23
1,1241032867699765249,2020-03-20,0,3,20,23
2,1241032875102703616,2020-03-20,1,3,20,25
3,1241032877099237379,2020-03-20,1,3,20,21
4,1241032870405128192,2020-03-20,0,3,20,23


In [126]:
    x = df1[['month', 'day' , 'Text_Id' , 'No. of words']]
    y = df1['Sentiment Score']
    different_models_performance(x , y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Score for Logistic Regression :  0.41428567025909224
Score for DecisionTreeClassifier :  0.7521257153775746


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Score for RandomForestClassifier :  0.7509114611423235


In [135]:
df.head()

,Text_Id,Text,Date,Location,Sentiments,Sentiment Score
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,1241032877099237379,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


In [136]:
df1.head()

,Text_Id,Date,Sentiment Score,month,day,No. of words
0,1241032866567356417,2020-03-20,0,3,20,23
1,1241032867699765249,2020-03-20,0,3,20,23
2,1241032875102703616,2020-03-20,1,3,20,25
3,1241032877099237379,2020-03-20,1,3,20,21
4,1241032870405128192,2020-03-20,0,3,20,23


In [137]:
df1['Contains_HashTag'] = df['Text']
df1.head()

,Text_Id,Date,Sentiment Score,month,day,No. of words,Contains_HashTag
0,1241032866567356417,2020-03-20,0,3,20,23,RT @theskindoctor13: Shaheen Bagh is still on....
1,1241032867699765249,2020-03-20,0,3,20,23,RT @theskindoctor13: Shaheen Bagh is still on....
2,1241032875102703616,2020-03-20,1,3,20,25,"RT @SmokingSkills_: Daughter of an IAS, son of..."
3,1241032877099237379,2020-03-20,1,3,20,21,RT @narendramodi: The young actors have someth...
4,1241032870405128192,2020-03-20,0,3,20,23,RT @theskindoctor13: Shaheen Bagh is still on....


In [157]:
df1['Contains_HashTag'] = df1['Contains_HashTag'].apply(lambda x : 0 if x.find('#') == -1 else 1 )
df1.head()

In [159]:
    x = df1[['month', 'day' , 'Text_Id' , 'No. of words' , 'Contains_HashTag']]
    y = df1['Sentiment Score']
    different_models_performance(x , y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Score for Logistic Regression :  0.41428567025909224
Score for DecisionTreeClassifier :  0.8126411878734834


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Score for RandomForestClassifier :  0.8117258744017333


In [160]:
df.head()

,Text_Id,Text,Date,Location,Sentiments,Sentiment Score
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,1241032877099237379,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


In [166]:
df1.head()

,Text_Id,Date,Sentiment Score,month,day,No. of words,Contains_HashTag
0,1241032866567356417,2020-03-20,0,3,20,23,0
1,1241032867699765249,2020-03-20,0,3,20,23,0
2,1241032875102703616,2020-03-20,1,3,20,25,0
3,1241032877099237379,2020-03-20,1,3,20,21,1
4,1241032870405128192,2020-03-20,0,3,20,23,0
